![Neptune + Great Expectations](https://neptune.ai/wp-content/uploads/2024/06/GreatExpectations.svg)

# Neptune + Great Expectations

<a target="_blank" href="https://colab.research.google.com/github/neptune-ai/examples/blob/main/integrations-and-supported-tools/great-expectations/notebooks/Neptune_Great_Expectations.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/>
</a><a target="_blank" href="https://github.com/neptune-ai/examples/blob/main/integrations-and-supported-tools/great-expectations/">
  <img alt="Open in GitHub" src="https://img.shields.io/badge/Open_in_GitHub-blue?logo=github&labelColor=black">
</a><a target="_blank" href="https://app.neptune.ai/o/showcase/org/great-expectations/runs/details?viewId=9c54e2be-0bd3-40cb-8868-08092ce30caf&detailsTab=dashboard&dashboardId=GX-metadata-9c54e2cf-4533-4b64-92a3-ae49ea174815">  
  <img alt="Explore in Neptune" src="https://neptune.ai/wp-content/uploads/2024/01/neptune-badge.svg">
</a><a target="_blank" href="https://docs.neptune.ai/integrations/great_expectations/">
  <img alt="View tutorial in docs" src="https://neptune.ai/wp-content/uploads/2024/01/docs-badge-2.svg">
</a>

## Introduction

[Great Expectations (GX) OSS](https://greatexpectations.io/gx-oss) is an open-source tool to help you validate, document, and monitor your data.
This guide will show you how to:

* Log GX OSS's configurations to Neptune,
* Log machine-readable validation results to Neptune,
* Upload GX OSS's interactive human-readable HTML reports to Neptune.

This notebook is a modified version of the [GX OSS quickstart](https://docs.greatexpectations.io/docs/oss/tutorials/quickstart) accessed on **2024 Jun 18**.

## Before you start

This notebook example lets you try out Neptune as an anonymous user, with zero setup.

If you want to see the example logged to your own workspace instead:

  1. Create a Neptune account. [Register &rarr;](https://neptune.ai/register)
  1. Create a Neptune project that you will use for tracking metadata. For instructions, see [Creating a project](https://docs.neptune.ai/setup/creating_project) in the Neptune docs.

## Install Neptune and GX OSS

In [ ]:
%pip install -q -U neptune great_expectations

## Import libraries

In [ ]:
import neptune
import great_expectations as gx

from neptune.utils import stringify_unsupported

### Start a Neptune run

To create a new run for tracking the metadata, you tell Neptune who you are (`api_token`) and where to send the data (`project`).

You can use the default code cell below to create an anonymous run in the public project [common/great-expectations](https://app.neptune.ai/o/common/org/great-expectations).

**Note**: Public projects are cleaned regularly, so anonymous runs are only stored temporarily.

### Log to your own project instead

Replace the code below with the following:

```python
import neptune
from getpass import getpass

run = neptune.init_run(
    project="workspace-name/project-name",  # Replace with your workspace and project names
    api_token=getpass("Enter your Neptune API token: "),
    tags=["notebook"],  # (Optional) Replace with your own tags
)
```

To find your API token and full project name:

1. [Log in to Neptune](https://app.neptune.ai/).
1. In the bottom-left corner, expand your user menu and select **Get your API token**.
1. To copy the project path, in the top-right corner, open the settings menu and select **Details & privacy**.

For more help, see [Setting Neptune credentials](https://docs.neptune.ai/setup/setting_credentials) in the Neptune docs.

In [ ]:
run = neptune.init_run(
    api_token=neptune.ANONYMOUS_API_TOKEN,
    project="common/great-expectations",
    tags=["notebook"],  # (optional) replace with your own
)

**To view the newly created run and its metadata in the Neptune app, use the link that appeared in the cell output.**

## Create a GX Data Context

In [ ]:
context = gx.get_context()

### Upload Context configuration to Neptune

In [ ]:
run["gx/context/config"] = context.get_config().to_json_dict()

The above code cell logs the GX data context configuration to the `gx/context/config` namespace in the Neptune run.

## Connect to data

In [ ]:
validator = context.sources.pandas_default.read_csv(
    "https://raw.githubusercontent.com/great-expectations/gx_tutorials/main/data/yellow_tripdata_sample_2019-01.csv"
)

## Create Expectations

In [ ]:
validator.expect_column_values_to_not_be_null("pickup_datetime")
validator.expect_column_values_to_be_between("passenger_count", min_value=1, max_value=6)
validator.save_expectation_suite(
    discard_failed_expectations=False,
    discard_catch_exceptions_kwargs=False,
    discard_include_config_kwargs=False,
    discard_result_format_kwargs=False,
)

### Log Expectations to Neptune

In [ ]:
expectation_suite = validator.get_expectation_suite().to_json_dict()

In [ ]:
run["gx/meta"] = expectation_suite["meta"]

In [ ]:
run["gx/expectations/expectations_suite_name"] = expectation_suite["expectation_suite_name"]

for idx, expectation in enumerate(expectation_suite["expectations"]):
    run["gx/expectations"][idx] = expectation

The above code cell does two things:
* logs the GX Expectations suite name of the context configuration to the `gx/expectations/expectations_suite_name` field of the run,
* creates a numbered folder for each expectation in the `gx/expectations` namespace.

## Create Checkpoint

In [ ]:
checkpoint = context.add_or_update_checkpoint(
    name="my_quickstart_checkpoint",
    validator=validator,
)

### Log Checkpoint config to Neptune

In [ ]:
run["gx/checkpoint/config"] = stringify_unsupported(checkpoint.config.to_json_dict())

Since the checkpoint configuration contains lists as values, we use [`stringify_unsupported()`](https://docs.neptune.ai/api/utils/#stringify_unsupported) to convert them to strings.

## Run Validations

In [ ]:
checkpoint_result = checkpoint.run()

### Log Validation results to Neptune

By saving GX OSS' results as a dictionary to Neptune, you can access them programmatically and use in your CI/CD pipelines.

In [ ]:
results_dict = checkpoint_result.list_validation_results()[0].to_json_dict()

In [ ]:
run["gx/validations/json"] = results_dict

In [ ]:
for idx, result in enumerate(results_dict["results"]):
    run["gx/validations/json/results"][idx] = result

## Upload HTML reports to Neptune

Using Neptune's HTML previewer, you can view and interact with GX OSS' rich HTML reports on Neptune.

### Get the `local_site_path` of the Data Context

In [ ]:
from great_expectations.data_context import EphemeralDataContext
import os

if isinstance(context, EphemeralDataContext):
    context = context.convert_to_file_context()

local_site_path = os.path.dirname(context.build_data_docs()["local_site"])[7:]

### Log Expectations reports to Neptune

In [ ]:
run["gx/expectations/reports"].upload_files(os.path.join(local_site_path, "expectations"))

### Log Validations reports to Neptune

In [ ]:
run["gx/validations/reports"].upload_files(os.path.join(local_site_path, "validations"))

## Cleanup

### Stop Neptune run

Once you are done logging, stop tracking the run.

In [ ]:
run.stop()

### (Optional) Delete the FileSystem Data Context

In [ ]:
import shutil

shutil.rmtree("gx")

## Analyze the logged metadata in the Neptune app
 
Explore the run in the Neptune app, or check this [example dashboard](https://app.neptune.ai/o/showcase/org/great-expectations/runs/details?viewId=9c54e2be-0bd3-40cb-8868-08092ce30caf&detailsTab=dashboard&dashboardId=GX-metadata-9c54e2cf-4533-4b64-92a3-ae49ea174815).